# 5.3 - Web Scraping (bs4)


![scraping](images/scraping.png)


Web scraping o raspado web, es una técnica utilizada mediante programas de software para extraer información de sitios web. Usualmente, estos programas simulan la navegación de un humano en la web ya sea utilizando el protocolo HTTP manualmente, o incrustando un navegador en una aplicación.

El web scraping está muy relacionado con la indexación de la web, la cual indexa la información de la web utilizando un robot y es una técnica universal adoptada por la mayoría de los motores de búsqueda. Sin embargo, el web scraping se enfoca más en la transformación de datos sin estructura en la web, como el formato HTML, en datos estructurados que pueden ser almacenados y analizados en una base de datos central, en una hoja de cálculo o en alguna otra fuente de almacenamiento. Alguno de los usos del web scraping son la comparación de precios en tiendas, la monitorización de datos relacionados con el clima de cierta región, la detección de cambios en sitios webs y la integración de datos en sitios webs. 

En los últimos años el web scraping se ha convertido en una técnica muy utilizada dentro del sector del posicionamiento web gracias a su capacidad de generar grandes cantidades de datos para crear contenidos de calidad.

Podríamos pensar que el web scraping es nuestro último recurso a falta de una API o un feed RSS. A falta de una fuente de datos, siempre podemos extraer aquello que sale por pantalla.

### Extracción desde el HTML

Para scrapear una página web, en primer lugar debemos conocer las estructura que tiene el HTML. Veamos la estructura básica.

El HTML consiste en contenido `<etiquetado>`, es como si fueran cajas de contenido, organizado de manera jerárquica:

```
<html>
    <head>
        <title>Titulo de la pagina</title>
    </head>
    <body>
        <h1>Cabecera</h1>
        <p>Parrafo</p>
    </body>
</html>
```

$$$$

Las etiquetas el HTML se pueden clasificar en varios grupos, dependiendo del tipo de contenido que posea. Estos son algunos ejemplos:

+ cabecera: `<h1>`, `<h2>`, `<h3>`, `<hgroup>`...
+ texto: `<b>`, `<p>`...
+ embebido: `<audio>`, `<img>`, `<video>`...
+ tabular: `<table>`, `<tr>`, `<td>`, `<tbody>`...
+ secciones: `<header>`, `<section>`, `<article>`...
+ metadata: `<meta>`, `<title>`, `<script>`...

$$$$

Las etiquetas pueden tener atributos. Por ejemplo:
 
`<div class="text-monospace" id="name_132", href="www.example.com"> Contenido de la pagina </div>` 

Esta etiqueta `div` tiene los siguientes atributos:

+ class: atributo con valor "text-monospace". La clase no es única en la página.
+ id: atributo con valor "name_132". El id de una etiqueta la identifica de manera unívoca.
+ href: atributo con valor "www.example.com". El href suele contener el link a otra parte de la página.

Siguiendo con la analogía de las cajas, si una etiqueta de HTML es una caja, los atributos serían las pegatinas pegadas en la tapa de la caja.

Conociendo cual es el contenido que queremos extraer, debemos encontrar las etiquetas dentro del HTML de la página web.

Usaremos la herramienta **[BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)**.

In [1]:
!pip install beautifulsoup4

In [2]:
import requests as req

from bs4 import BeautifulSoup as bs

### Ejemplos Wikipedia

**[Países europeos según esperanza de vida](https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy)**

In [3]:
url='https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy'

In [24]:
# usamos requests para extraer el HTML

html=req.get(url).content # o .text

html[:1000]

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of European countries by life expectancy - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"91d84c4c-b45d-412c-8702-5972647193b2","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_European_countries_by_life_expectancy","wgTitle":"List of European countries by life expectancy","wgCurRevisionId":1049636953,"wgRevisionId":1049636953,"wgArticleId":22175559,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is differen

In [25]:
# bs4 para el manejo del html

soup=bs(html, 'html.parser')

type(soup)

bs4.BeautifulSoup

In [26]:
# unica tabla en la pagina

tabla=soup.find('table')  # find saca solo el primero, devuelve elemento

type(tabla)

bs4.element.Tag

In [27]:
# extraccion de las filas de la tabla

filas=tabla.find_all('tr')   # find_all saca todos, devuelve una lista

filas=[f.text.strip().split('\n') for f in filas]

filas[:5]

[['Rank',
  '',
  'Country',
  'Life expectancy[1]',
  '',
  'Influenza vaccination rate, people aged 65 and over, 2016 (%)[2]'],
 ['1', '', '\xa0Monaco[3]', '', '89.4'],
 ['2', '', '\xa0San Marino[4]', '', '83.4'],
 ['3', '', '\xa0\xa0Switzerland', '83.0'],
 ['4', '', '\xa0Spain', '82.8', '', '56%']]

In [28]:
# minima limpieza, quitar strings vacias

final=[]

for fila in filas:
    
    tmp=[]
    
    for palabra in fila:
        
        if palabra!='':
            
            tmp.append(palabra)
            
    final.append(tmp)
    
final[:5]

[['Rank',
  'Country',
  'Life expectancy[1]',
  'Influenza vaccination rate, people aged 65 and over, 2016 (%)[2]'],
 ['1', '\xa0Monaco[3]', '89.4'],
 ['2', '\xa0San Marino[4]', '83.4'],
 ['3', '\xa0\xa0Switzerland', '83.0'],
 ['4', '\xa0Spain', '82.8', '56%']]

In [29]:
import pandas as pd

col_names=final[0]

data=final[1:]

df=pd.DataFrame(data, columns=col_names)

df.head(10)

,Rank,Country,Life expectancy[1],"Influenza vaccination rate, people aged 65 and over, 2016 (%)[2]"
0,1,Monaco[3],89.4,None
1,2,San Marino[4],83.4,None
2,3,Switzerland,83.0,None
3,4,Spain,82.8,56%
4,5,Liechtenstein,82.7,28%
5,6,Italy,82.7,50%
6,7,Norway,82.5,38%
7,8,Iceland,82.5,47%
8,9,Luxembourg,82.3,38%
9,10,France,82.3,50%


$$$$

**[Medallero Barcelona'92](https://es.wikipedia.org/wiki/Juegos_Ol%C3%ADmpicos_de_Barcelona_1992)**

In [30]:
url='https://es.wikipedia.org/wiki/Juegos_Ol%C3%ADmpicos_de_Barcelona_1992' # 1er paso, url

In [31]:
html=req.get(url).text   # 2º paso, html

In [32]:
soup=bs(html, 'html.parser')   # 3er paso sopa, y ya tenemos html

In [36]:
tabla=soup.find_all('table')[-1]

In [37]:
elem=tabla.find('a')

elem  # caja tag a

<a href="/wiki/Control_de_autoridades" title="Control de autoridades">Control de autoridades</a>

In [38]:
elem.text   # contenido de la caja

'Control de autoridades'

In [40]:
elem.contents   # contenido como lista

['Control de autoridades']

In [41]:
elem.attrs  # stickers en la tapa de la caja

{'href': '/wiki/Control_de_autoridades', 'title': 'Control de autoridades'}

In [42]:
 elem.attrs['href']

'/wiki/Control_de_autoridades'

In [43]:
medallero=soup.find_all('table')[-4]

In [53]:
med_paises=[]

for f in medallero.find_all('tr'):  # bucle por filas de la tabla
    
    fila=[e for e in f.find_all('td')]
    
    if len(fila)>0:
        
        pais={
            'nombre': fila[1].find('a').text.strip(),
            'oros': int(fila[2].text),
            'platas': int(fila[3].text),
            'bronces': int(fila[4].text),
            'total': int(fila[5].text)
        }
        
        med_paises.append(pais)
        
med_paises

[{'nombre': 'Equipo Unificado',
  'oros': 45,
  'platas': 38,
  'bronces': 29,
  'total': 112},
 {'nombre': 'Estados Unidos',
  'oros': 37,
  'platas': 34,
  'bronces': 37,
  'total': 108},
 {'nombre': 'Alemania', 'oros': 33, 'platas': 21, 'bronces': 28, 'total': 82},
 {'nombre': 'China', 'oros': 16, 'platas': 22, 'bronces': 16, 'total': 54},
 {'nombre': 'Cuba', 'oros': 14, 'platas': 6, 'bronces': 11, 'total': 31},
 {'nombre': 'España', 'oros': 13, 'platas': 7, 'bronces': 2, 'total': 22},
 {'nombre': 'Corea del Sur',
  'oros': 12,
  'platas': 5,
  'bronces': 12,
  'total': 29},
 {'nombre': 'Hungría', 'oros': 11, 'platas': 12, 'bronces': 7, 'total': 30},
 {'nombre': 'Francia', 'oros': 8, 'platas': 5, 'bronces': 16, 'total': 29},
 {'nombre': 'Australia', 'oros': 7, 'platas': 9, 'bronces': 11, 'total': 27}]

In [54]:
pd.DataFrame(med_paises)

,nombre,oros,platas,bronces,total
0,Equipo Unificado,45,38,29,112
1,Estados Unidos,37,34,37,108
2,Alemania,33,21,28,82
3,China,16,22,16,54
4,Cuba,14,6,11,31
5,España,13,7,2,22
6,Corea del Sur,12,5,12,29
7,Hungría,11,12,7,30
8,Francia,8,5,16,29
9,Australia,7,9,11,27


$$$$

**[Videoconsolas](https://es.wikipedia.org/wiki/Videoconsola)**

In [55]:
url='https://es.wikipedia.org/wiki/Videoconsola'

In [57]:
html=req.get(url).text

soup=bs(html, 'html.parser')

In [58]:
tablas=soup.find_all('table')

tabla=tablas[-2]

In [59]:
filas=tabla.find_all('tr')

filas=[f.text.strip().split('\n') for f in filas]

final=[]

for e in filas:
    tmp=[]
    
    for st in e:
        if st!='':
            tmp.append(st)
            
    final.append(tmp)
    
final[:3]

[['Fabricante', 'Consola', 'Lanzamiento', 'Unidades vendidas'],
 ['Nintendo',
  'Nintendo DS',
  '11 de marzo de 2005',
  '5.7\xa0millones (hasta 2012)[63]\u200b'],
 ['Sony',
  'PlayStation 2',
  '24 de noviembre de 2000',
  '5\xa0millones (hasta 2009)[64]\u200b']]

In [61]:
df=pd.DataFrame(final[1:], columns=final[0])

df

,Fabricante,Consola,Lanzamiento,Unidades vendidas
0,Nintendo,Nintendo DS,11 de marzo de 2005,5.7 millones (hasta 2012)[63]​
1,Sony,PlayStation 2,24 de noviembre de 2000,5 millones (hasta 2009)[64]​
2,Sony,PlayStation 4,29 de noviembre de 2013,3.3 millones (hasta 2019)[65]​
3,Nintendo,Wii,8 de diciembre de 2006,2.7 millones (hasta 2011)[66]​
4,Sony,PlayStation 3,23 de marzo de 2007,2.7 millones (hasta 2014)[67]​
5,Nintendo,Game Boy,Enero de 1991[68]​,2.3 millones (hasta 1999)[69]​
6,Sony,PlayStation Portable,1 de septiembre de 2005,2.1 millones (hasta 2009)[70]​
7,Nintendo,Nintendo 3DS,25 de marzo de 2011,2 millones (hasta 2018)[71]​
8,Nintendo,Game Boy Advance,22 de junio de 2001[72]​,1.6 millones (hasta 2004)[73]​
9,Nintendo,Switch,3 de marzo de 2017,1.57 millones (hasta 2020)[74]​


In [63]:
df.Lanzamiento=df.Lanzamiento.apply(lambda x: int(x.split('[')[0][-4:]))

df.head()

,Fabricante,Consola,Lanzamiento,Unidades vendidas
0,Nintendo,Nintendo DS,2005,5.7 millones (hasta 2012)[63]​
1,Sony,PlayStation 2,2000,5 millones (hasta 2009)[64]​
2,Sony,PlayStation 4,2013,3.3 millones (hasta 2019)[65]​
3,Nintendo,Wii,2006,2.7 millones (hasta 2011)[66]​
4,Sony,PlayStation 3,2007,2.7 millones (hasta 2014)[67]​


### Ejemplo geolocalización por IP

https://tools.keycdn.com/geo

**¿Dónde estoy?**

In [64]:
url='https://tools.keycdn.com/geo'

In [66]:
html=req.get(url).text

soup=bs(html, 'html.parser')

In [68]:
soup.find('div', id='geoResult')

<div class="mt-4" id="geoResult">
<div class="bg-light medium rounded p-3">
<p class="small text-uppercase text-muted font-weight-semi-bold line-height-headings mb-2">Location</p> <dl class="row mb-0">
<dt class="col-4">City</dt><dd class="col-8 text-monospace">Humanes de Madrid</dd><dt class="col-4">Region</dt><dd class="col-8 text-monospace">Madrid (M)</dd><dt class="col-4">Postal code</dt><dd class="col-8 text-monospace">28970</dd><dt class="col-4">Country</dt><dd class="col-8 text-monospace">Spain (ES)</dd><dt class="col-4">Continent</dt><dd class="col-8 text-monospace">Europe (EU)</dd><dt class="col-4">Coordinates</dt><dd class="col-8 text-monospace">40.2493 (lat) / -3.8357 (long)</dd><dt class="col-4">Time</dt><dd class="col-8 text-monospace">2021-11-13 11:40:33 (Europe/Madrid)</dd> </dl>
<p class="small text-uppercase text-muted font-weight-semi-bold line-height-headings mt-4 mb-2">Network</p>
<dl class="row mb-0">
<dt class="col-4">IP address</dt><dd class="col-8 text-monospace

In [70]:
data=soup.find('div', id='geoResult')

conexion=[e.text for e in data.find_all('dd', class_="col-8 text-monospace")]

conexion

['Humanes de Madrid',
 'Madrid (M)',
 '28970',
 'Spain (ES)',
 'Europe (EU)',
 '40.2493 (lat) / -3.8357 (long)',
 '2021-11-13 11:40:33 (Europe/Madrid)',
 '213.99.35.30',
 '213.99.35.30',
 'Telefonica De Espana',
 '3352']

**Búsqueda según IP**

https://tools.keycdn.com/geo?host=137.255.90.7

In [71]:
ip='137.255.90.7'

url=f'https://tools.keycdn.com/geo?host={ip}'

In [72]:
html=req.get(url).text

soup=bs(html, 'html.parser')

In [73]:
data=soup.find('div', id='geoResult')

conexion=[e.text for e in data.find_all('dd', class_="col-8 text-monospace")]

conexion

['Benin (BJ)',
 'Africa (AF)',
 '9.5 (lat) / 2.25 (long)',
 '2021-11-13 11:45:27 (Africa/Porto-Novo)',
 '137.255.90.7',
 '137.255.90.7']

In [74]:
def findme(ip):
    url=f'https://tools.keycdn.com/geo?host={ip}'
    html=req.get(url).text
    soup=bs(html, 'html.parser')
    data=soup.find('div', id='geoResult')
    conexion=[e.text for e in data.find_all('dd', class_="col-8 text-monospace")]
    return conexion

In [76]:
findme('137.255.90.10')

['Benin (BJ)',
 'Africa (AF)',
 '9.5 (lat) / 2.25 (long)',
 '2021-11-13 11:46:37 (Africa/Porto-Novo)',
 '137.255.90.10',
 '137.255.90.10']

### Ejemplo LinkedIn